# Time Series Analysis in Public Health

### Packages
- ISOweek url: https://cran.r-project.org/web/packages/ISOweek/ISOweek.pdf (https://cran.r-project.org/web/packages/ISOweek/ISOweek.pdf) 

- Linear model applied to daily mortality
- 1991 heat wave (data from 1989 to 1993)

###  Set working directory

In [38]:
getwd()  # Prints the current working directory

[1] "/home/nbuser/library"

In [39]:
install.packages("season")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [40]:
dados <- read.csv("/home/nbuser/library/Portugal_GC_CR.csv", header = T, sep = ";")
str(dados)

'data.frame':	12054 obs. of  3 variables:
 $ Data    : Factor w/ 12054 levels "01-01-1980","01-01-1981",..: 1 397 793 1189 1585 1981 2377 2773 3169 3565 ...
 $ Total   : int  387 340 312 278 258 301 298 302 331 281 ...
 $ CircResp: int  186 166 180 153 132 164 166 156 170 136 ...


### need to correct the Date variable for the Data type

In [41]:
dados$Data <- as.Date(dados$Data,format="%d-%m-%Y")

### View the time series


In [42]:
plot(dados$Total~dados$Data,
     type="l",
     main="Time series of \n all-cause daily mortality",
     ylab="Number of deaths",
     xlab="Date")
points(x = subset(dados,dados$Total==max(dados$Total))$Data,
       y=subset(dados,dados$Total==max(dados$Total))$Total,col="red",pch=4)
text(x = subset(dados,dados$Total==max(dados$Total))$Data+1500,
     y=subset(dados,dados$Total==max(dados$Total))$Total,
     labels = paste0(max(dados$Total)," (",subset(dados,dados$Total==max(dados$Total))$Data,")"),cex=0.8)

###  We will restrict the series between 1989 and 1993 to estimate the excesses associated with the heat wave later

In [43]:
dados <- subset(dados,dados$Data>=as.Date("1989-01-01") &
                  dados$Data<=as.Date("1993-12-31"))

### see first and last records of the new series to confirm that it was restricted

In [44]:
head(dados,3)

,Data,Total,CircResp
,<date>,<int>,<int>
3289,1989-01-01,374,230
3290,1989-01-02,379,229
3291,1989-01-03,363,224


In [45]:
tail(dados,3)

,Data,Total,CircResp
,<date>,<int>,<int>
5112,1993-12-29,387,235
5113,1993-12-30,396,232
5114,1993-12-31,374,221


###  create the variable time

In [46]:
dados$t <- seq(1,nrow(dados),1)

In [48]:
plot(dados$Total~dados$Data,type="l",main="Daily mortality in Portugal \n 1989 to 1993",
     ylim=c(0,max(dados$Total)),ylab="death",xlab="Data",las=1)

lines(dados$CircResp~dados$Data,col="orange")
legend("bottomleft",legend = c("TC", "CircResp"),text.col = c(1,"orange"),horiz = F,bty = "n",cex = 0.8)

### using a Serfling Model quoted in many articles
### Serfling Model - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1915276/

In [49]:
t <- seq(1,nrow(dados),1)
t2 <- t^2
t3 <- t^3
sin365 <- sin(2*pi*t/365)
cos365 <- cos(2*pi*t/365)

###  confirmation of the period to use

In [50]:
library(season)
modTen <- lm(dados$Total ~ t)

### summary(modTen)

In [51]:
p <- data.frame(peri(dados$Total-modTen$fitted.values,adjmean=TRUE,plot=TRUE))
p[order(-p$amp),][1:5,c("c","amp")] # ver os cinco periodos com maior energia (+/- 1 ano e 6 meses)

,c,amp
,<dbl>,<dbl>
6,365.2000,51.593477
11,182.6000,25.951529
9,228.2500,11.529539
21,91.3000,8.831709
14,140.4615,8.431515


In [52]:
modTen <- lm(dados$CircResp ~ t)
summary(modTen)


Call:
lm(formula = dados$CircResp ~ t)

Residuals:
    Min      1Q  Median      3Q     Max 
-63.196 -26.843  -8.748  20.173 139.654 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.406e+02  1.716e+00  81.936   <2e-16 ***
t           1.956e-03  1.627e-03   1.202    0.229    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 36.64 on 1824 degrees of freedom
Multiple R-squared:  0.0007916,	Adjusted R-squared:  0.0002438 
F-statistic: 1.445 on 1 and 1824 DF,  p-value: 0.2295


In [54]:
p <- data.frame(peri(dados$CircResp-modTen$fitted.values,adjmean=TRUE,plot=TRUE))
p[order(-p$amp),][1:5,c("c","amp")] # see the five periods with the highest energy (+/- 1 year and 6 months)

,c,amp
,<dbl>,<dbl>
6,365.2000,39.266734
11,182.6000,17.021362
9,228.2500,7.766540
19,101.4444,5.977408
21,91.3000,5.949101


In [55]:
mod <- lm(dados$CircResp~ t + t2 + t3 + sin365+cos365)
summary(mod)


Call:
lm(formula = dados$CircResp ~ t + t2 + t3 + sin365 + cos365)

Residuals:
    Min      1Q  Median      3Q     Max 
-51.080 -15.687  -3.149  12.076 150.981 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.250e+02  2.263e+00  55.261  < 2e-16 ***
t            7.831e-02  1.073e-02   7.296 4.42e-13 ***
t2          -9.444e-05  1.365e-05  -6.919 6.26e-12 ***
t3           3.335e-08  4.914e-09   6.787 1.54e-11 ***
sin365       1.971e+01  8.097e-01  24.342  < 2e-16 ***
cos365       3.488e+01  7.794e-01  44.756  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 23.54 on 1820 degrees of freedom
Multiple R-squared:  0.5886,	Adjusted R-squared:  0.5875 
F-statistic: 520.9 on 5 and 1820 DF,  p-value: < 2.2e-16


### New data

In [56]:
newdata<-data.frame(t=t,t2=t2,t3=t3,sin365=sin365,cos365=cos365)
head(newdata)

t,t2,t3,sin365,cos365
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,0.01721336,0.9998518
2,4,8,0.03442161,0.9994074
3,9,27,0.05161967,0.9986668
4,16,64,0.06880243,0.9976303
5,25,125,0.08596480,0.9962982
6,36,216,0.10310170,0.9946708


In [57]:
esperado <- data.frame("fit"=predict(mod,newdata,interval="prediction",level = 0.95)[,1],
                       "lwr95"=predict(mod,newdata,interval="prediction",level = 0.95)[,2],
                       "upr95"=predict(mod,newdata,interval="prediction",level = 0.95)[,3],
                       "lwr99"=predict(mod,newdata,interval="prediction",level = 0.99)[,2],
                       "upr99"=predict(mod,newdata,interval="prediction",level = 0.99)[,3])
head(esperado)

fit,lwr95,upr95,lwr99,upr99
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
160.3372,113.9412,206.7332,99.3391,221.3353
160.7389,114.3452,207.1326,99.7438,221.7339
161.1298,114.7384,207.5213,100.1377,222.1219
161.5100,115.1208,207.8992,100.5209,222.4991
161.8792,115.4923,208.2662,100.8930,222.8654
162.2375,115.8527,208.6222,101.2542,223.2208


In [58]:
plot(dados$CircResp~dados$Data,type="l",main="Daily Mortality in Portugal \n CircResp \n 1989 to 1993",
     ylab="Deaths",xlab="Date",las=1,ylim=c(0,max(dados$CircResp)))
lines(esperado$fit~dados$Data,col="blue",lwd=3)
lines(esperado$upr95~dados$Data,col="orange",lwd=2)
lines(esperado$upr99~dados$Data,col="red",lwd=2)
lines(esperado$lwr95~dados$Data,col="green",lwd=2)
lines(esperado$lwr99~dados$Data,col="darkgreen",lwd=2)

### we will join the original base (observed) with the expected ones
### to see which days are above 95% confidence to withdraw from the series

In [59]:
base <- data.frame("Data"=dados$Data,"CircResp"=dados$CircResp,
                   "Esperado"=esperado$fit,"LimiteSuperior95"=esperado$upr95)

head(base)

Data,CircResp,Esperado,LimiteSuperior95
<date>,<int>,<dbl>,<dbl>
1989-01-01,230,160.3372,206.7332
1989-01-02,229,160.7389,207.1326
1989-01-03,224,161.1298,207.5213
1989-01-04,203,161.5100,207.8992
1989-01-05,209,161.8792,208.2662
1989-01-06,217,162.2375,208.6222


### Create a new variable where we take the days whose observations were higher than the upper limit 95%

In [60]:
base$CircResp_new <- ifelse(base$CircResp<base$LimiteSuperior95,
                            base$CircResp,NA)

### Adjust model to new series

In [61]:
mod1 <- lm(base$CircResp_new ~ t + t2 + t3 + sin365+cos365)
summary(mod1)

esperado1 <- data.frame("fit"=predict(mod1,newdata,interval="prediction",level = 0.95)[,1],
                       "lwr95"=predict(mod1,newdata,interval="prediction",level = 0.95)[,2],
                       "upr95"=predict(mod1,newdata,interval="prediction",level = 0.95)[,3],
                       "lwr99"=predict(mod1,newdata,interval="prediction",level = 0.99)[,2],
                       "upr99"=predict(mod1,newdata,interval="prediction",level = 0.99)[,3])
head(esperado1)


Call:
lm(formula = base$CircResp_new ~ t + t2 + t3 + sin365 + cos365)

Residuals:
    Min      1Q  Median      3Q     Max 
-48.616 -12.835  -1.239  11.696  54.347 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.231e+02  1.863e+00  66.085  < 2e-16 ***
t            5.674e-02  8.872e-03   6.395 2.06e-10 ***
t2          -6.089e-05  1.131e-05  -5.385 8.22e-08 ***
t3           2.091e-08  4.080e-09   5.125 3.31e-07 ***
sin365       1.931e+01  6.485e-01  29.783  < 2e-16 ***
cos365       3.268e+01  6.456e-01  50.628  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 18.65 on 1731 degrees of freedom
  (89 observations deleted due to missingness)
Multiple R-squared:  0.6637,	Adjusted R-squared:  0.6627 
F-statistic: 683.2 on 5 and 1731 DF,  p-value: < 2.2e-16


fit,lwr95,upr95,lwr99,upr99
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
156.1503,119.3671,192.9336,107.7895,204.5112
156.5247,119.7435,193.3060,108.1664,204.8830
156.8891,120.1098,193.6684,108.5334,205.2449
157.2434,120.4660,194.0208,108.8902,205.5966
157.5875,120.8120,194.3630,109.2368,205.9382
157.9214,121.1478,194.6950,109.5731,206.2696


### joining the bases we have

In [62]:
base1 <- data.frame("Data"=dados$Data,"CircResp"=dados$CircResp,
                    "Esperado"=esperado1$fit,"LimiteSuperior95"=esperado1$upr95,
                    "LimiteSuperior99"=esperado1$upr99)

head(base1)
base1$excesso <- ifelse(base1$CircResp<base1$LimiteSuperior95,
                        0,base1$CircResp-base1$Esperado)

Data,CircResp,Esperado,LimiteSuperior95,LimiteSuperior99
<date>,<int>,<dbl>,<dbl>,<dbl>
1989-01-01,230,156.1503,192.9336,204.5112
1989-01-02,229,156.5247,193.3060,204.8830
1989-01-03,224,156.8891,193.6684,205.2449
1989-01-04,203,157.2434,194.0208,205.5966
1989-01-05,209,157.5875,194.3630,205.9382
1989-01-06,217,157.9214,194.6950,206.2696


In [64]:
plot(base1$CircResp~base1$Data,type="l",main="Daily Mortality in Portugal \n CircResp \n 1989 to 1993",
     ylab="Deaths",xlab="Date",las=1,ylim=c(-70,300),axes=F)
axis.Date(1,at=seq(min(base1$Data),max(base1$Data)+1,by="6 months"),format="%m-%Y",cex.axis=0.6)
axis(2,seq(0,300,50))
lines(esperado1$fit~base1$Data,col="blue",lwd=3,lty=2)
lines(esperado1$upr95~base1$Data,col="orange",lwd=2,lty=2)
lines(esperado1$upr99~base1$Data,col="red",lwd=2,lty=2)
lines(esperado1$lwr95~base1$Data,col="green",lwd=2,lty=2)
lines(esperado1$lwr99~base1$Data,col="darkgreen",lwd=2,lty=2)
par(new=T)
plot(base1$excesso~base1$Data,type="h",ylim=c(0,400),col="grey",axes=F,ylab="",xlab="")
axis(4,seq(0,160,20),col="grey")

### as what matters to us and the 1991 heat wave that took place in July we will restrict to this period

In [65]:
base2 <- subset(base1,base1$Data>="1991-07-01" & base1$Data<="1991-07-31")

### you save the base and then work it in excel for calculating the excesses themselves
#write.csv2(base1,"Julho1991.csv",row.names = F)

In [66]:
base2$Excesso <- ifelse(base2$CircResp>base2$LimiteSuperior95,floor(base2$CircResp-base2$Esperado),NA)

### Total excess

In [67]:
sum(base2$Excesso,na.rm=T)

[1] 697

### plot of the month of july

In [69]:
plot(base2$CircResp~base2$Data,type="l",main="Daily mortality in Portugal \n CircResp \n (data from 1989 to 1993)",
     ylab="Deaths",xlab="July of 1991",las=1,ylim=c(-70,300),axes=F)
axis.Date(1,at=seq(min(base2$Data),max(base2$Data)+1,by="3 days"),format="%d-%m",cex.axis=0.6)
axis(2,seq(0,300,50))
lines(base2$Esperado~base2$Data,col="blue",lwd=3)
lines(base2$LimiteSuperior95~base2$Data,col="orange",lwd=2)
lines(base2$LimiteSuperior99~base2$Data,col="red",lwd=2)
par(new=T)
plot(base2$Excesso~base2$Data,type="h", lwd = 6,ylim=c(0,400),col="grey",axes=F,ylab="",xlab="")
axis(4,seq(0,160,20),col="grey")

### remove all existing objects to start new analysis

In [37]:
rm(list=ls())

In [ ]:
##### Modelo Linear aplicado a mortalidade semanal ######
####### Estudo da epidemia de gripe de  1998/99 #########
############### dados entre 1995 e 2002 #################

In [ ]:
# Definir directoria do trabalho

In [ ]:
setwd("~/Desktop/INSA")

In [ ]:
# Ler o ficheiro de dados formato .csv

In [ ]:
dados <- read.csv("Portugal_GC_CR.csv", header = T, sep = ";")
str(dados)

In [ ]:
#so precisamos corrigir a variavel Data para o tipo Data

In [ ]:
dados$Data <- as.Date(dados$Data,format="%d-%m-%Y")

In [ ]:
#criar a variavel Semana

In [ ]:
library(ISOweek)
dados$ISOweek <- ISOweek(dados$Data)

In [ ]:
# Vamos restringir a serie entre entre 1995 e 2002 para estimarmos os excessos da epidemia posteriormente

In [ ]:
dados <- subset(dados,dados$ISOweek>=ISOweek("1995-01-01") &
                  dados$ISOweek<=ISOweek("2002-12-31"))

In [ ]:
#ver primeiros e ultimos registos da nova serie a confirmar que foi restringida
```{r}
head(dados,3);tail(dados,3)
```


#Criar a base semanal
```{r}
dados <- aggregate(x = dados[,c("Total","CircResp")],
                   by = list(dados$ISOweek),sum)
head(dados)
colnames(dados) <- c("ISOweek","Total","CircResp")
```

#criar a variavel tempo
```{r}
dados$t <- seq(1,nrow(dados),1)
dados$Data <- ISOweek2date(paste0(dados$ISOweek,"-1"))

plot(dados$Total~dados$Data,type="l",main="Mortalidade semanal em Portugal \n 1995 a 2002",
     ylim=c(0,max(dados$Total)),ylab="Obitos",xlab="Semana",las=1,axes="F")
axis(2,cex.axis=0.7,las=1)
axis.Date(1,at=seq(min(dados$Data),max(dados$Data),by="6 months"),format="%V-%Y",cex.axis=0.7,las=2)
lines(dados$CircResp~dados$Data,col="orange")
legend("bottomleft",legend = c("TC", "CircResp"),text.col = c(1,"orange"),horiz = F,bty = "n",cex = 0.8)
```


#Serfling Model - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1915276/

```{r}
t <- seq(1,nrow(dados),1)
t2 <- t^2
t3 <- t^3
sin52 <- sin(2*pi*t/52)
cos52 <- cos(2*pi*t/52)
```



```{r}
library(season)
modTen <- lm(dados$CircResp ~ t)
summary(modTen)

```
```{r}
p <- data.frame(peri(dados$CircResp-modTen$fitted.values,adjmean=TRUE,plot=TRUE))
p[order(-p$amp),][1:5,c("c","amp")] # ver os cinco periodos com maior energia (+/- 1 ano e 6 meses)
```


```{r}
mod <- lm(dados$CircResp~ t + t2 + t3 + sin52+cos52)
summary(mod)
```


#novos dados
```{r}
newdata<-data.frame(t=t,t2=t2,t3=t3,sin52=sin52,cos52=cos52)
head(newdata)

```


```{r}
esperado <- data.frame("fit"=predict(mod,newdata,interval="prediction",level = 0.95)[,1],
                       "lwr95"=predict(mod,newdata,interval="prediction",level = 0.95)[,2],
                       "upr95"=predict(mod,newdata,interval="prediction",level = 0.95)[,3],
                       "lwr99"=predict(mod,newdata,interval="prediction",level = 0.99)[,2],
                       "upr99"=predict(mod,newdata,interval="prediction",level = 0.99)[,3])
head(esperado)
```


```{r}
plot(dados$CircResp~dados$Data,type="l",main="Mortalidade Semanal em Portugal \n CircResp \n1995 a 2002",
     ylab="Obitos",xlab="Semana",las=1,ylim=c(0,max(dados$CircResp)))

lines(esperado$fit~dados$Data,col="blue",lwd=3)
lines(esperado$upr95~dados$Data,col="orange",lwd=2)
lines(esperado$upr99~dados$Data,col="red",lwd=2)
lines(esperado$lwr95~dados$Data,col="green",lwd=2)
lines(esperado$lwr99~dados$Data,col="darkgreen",lwd=2)
```



#vamos juntar a base original (observados) com os esperados
#para ver quais os dias acima dos 95 % de confiança para retiramos da serie
```{r}
base <- data.frame("ISOweek"=dados$ISOweek,"Data"=dados$Data,"CircResp"=dados$CircResp,
                   "Esperado"=esperado$fit,"LimiteSuperior95"=esperado$upr95)

head(base)
```


#Criar uma nova variavel onde retiramos os dias cujas observaçoes foram superiores ao Limite superior 95%
```{r}
base$CircResp_new <- ifelse(base$CircResp<base$LimiteSuperior95,
                            base$CircResp,NA)

```

# Ajustar modelo a nova serie

```{r}
mod1 <- lm(base$CircResp_new ~ t + t2 + t3 + sin52 + cos52)
summary(mod1)

esperado1 <- data.frame("fit"=predict(mod1,newdata,interval="prediction",level = 0.95)[,1],
                        "lwr95"=predict(mod1,newdata,interval="prediction",level = 0.95)[,2],
                        "upr95"=predict(mod1,newdata,interval="prediction",level = 0.95)[,3],
                        "lwr99"=predict(mod1,newdata,interval="prediction",level = 0.99)[,2],
                        "upr99"=predict(mod1,newdata,interval="prediction",level = 0.99)[,3])

```

```{r}
head(esperado1)
```

#juntando as bases temos:

```{r}
base1 <- data.frame("ISOweek"=dados$ISOweek, "Data"=dados$Data,"CircResp"=dados$CircResp,
                    "Esperado"=esperado1$fit,"LimiteSuperior95"=esperado1$upr95,
                    "LimiteSuperior99"=esperado1$upr99)

head(base1)
base1$excesso <- ifelse(base1$CircResp<base1$LimiteSuperior95,
                        NA,base1$CircResp-base1$Esperado)


```


```{r}
plot(base1$CircResp~base1$Data,type="l",main="Mortalidade semanal em Portugal \n CircResp \n1994 a 2002",
     ylab="Obitos",xlab="Semana",las=1,ylim=c(0,2000),axes=F)
axis.Date(1,at=seq(min(base1$Data),max(base1$Data)+1,by="6 months"),format="%V-%Y",cex.axis=0.6,las=2)
axis(2,seq(0,2000,500),las=1,cex.axis=0.6)
lines(esperado1$fit~base1$Data,col="blue",lwd=3,lty=2)
lines(esperado1$upr95~base1$Data,col="orange",lwd=2,lty=2)
lines(esperado1$upr99~base1$Data,col="red",lwd=2,lty=2)
lines(esperado1$lwr95~base1$Data,col="green",lwd=2,lty=2)
lines(esperado1$lwr99~base1$Data,col="darkgreen",lwd=2,lty=2)
par(new=T)
plot(base1$excesso~base1$Data,type="h",ylim=c(0,2000),col="grey",axes=F,ylab="",xlab="")
axis(4,seq(0,600,200),col="grey",cex.axis=0.6)
```


# como o que nos importa e a epidemia de 98/99 vamos 
# restringir a esse periodo
#sugiro que guardem a base e depois a trabalhem em excel para o calculo dos excessos propriamente ditos

```{r}
base2 <- subset(base1,base1$Data>=ISOweek2date("1998-W40-1") & base1$Data<=ISOweek2date("1999-W20-1"))
base2

```

#sugiro que guardem a base e depois a trabalhem em excel para o calculo dos excessos propriamente ditos
#write.csv2(base1,"Epoca1998_99.csv",row.names = F)


#Total de excesso
```{r}
floor(sum(base2$excesso,na.rm=T))
```



#plot da epoca

```{r}
plot(base2$CircResp~base2$Data,type="l",main="Mortalidade semanal em Portugal \n CircResp \n (Epoca 1998/99)",
     ylab="Obitos",xlab="Semana",las=1,ylim=c(0,2000),axes=F)
axis.Date(1,at=seq(min(base2$Data),max(base2$Data)+1,by="4 weeks"),format="%V-%Y",cex.axis=0.6)
axis(2,seq(0,2000,500),cex.axis=0.6,las=1)
lines(base2$Esperado~base2$Data,col="blue",lwd=3)
lines(base2$LimiteSuperior95~base2$Data,col="orange",lwd=2)
lines(base2$LimiteSuperior99~base2$Data,col="red",lwd=2)
par(new=T)
plot(base1$excesso~base1$Data,type="h", lwd = 2,ylim=c(0,2000),col="grey",axes=F,ylab="",xlab="")
axis(4,seq(0,800,200),col="grey",cex.axis=0.6,las=1)

```


##### Suponhamos agora que queremos saber o que esperamos num futuro próximo tendo em conta o historico.
##### Vamos prever usando novos dados relativos ao tempo futuro
##### Todas as causas vai ser o mais rapido de obter 

#ver o modelo que temos
```{r}
summary(mod1)
```


#definir o periodo total para o qual pretendemos obter valores estimados pelo modelo
```{r}
t <- seq(1,nrow(dados)+52,1) # vamos estimar um corredor para mais um ano (52 semanas)
t2 <- t^2
t3 <- t^3
sin52 <- sin(2*pi*t/52)
cos52 <- cos(2*pi*t/52)
```


#novos dados
```{r}
newdata<-data.frame(t=t,t2=t2,t3=t3,sin52=sin52,cos52=cos52)
head(newdata)
tail(newdata)

esperado <- data.frame("fit"=predict(mod1,newdata,interval="prediction",level = 0.95)[,1],
                       "lwr95"=predict(mod1,newdata,interval="prediction",level = 0.95)[,2],
                       "upr95"=predict(mod1,newdata,interval="prediction",level = 0.95)[,3],
                       "upr95_CI"=predict(mod1,newdata,interval="confidence",level = 0.95)[,3],
                       "lwr99"=predict(mod1,newdata,interval="prediction",level = 0.99)[,2],
                       "upr99"=predict(mod1,newdata,interval="prediction",level = 0.99)[,3],
                       "upr99_CI"=predict(mod1,newdata,interval="confidence",level = 0.99)[,3])
head(esperado)

```

##adicionar indicativo das semanas e dias a base esperados
```{r}
esperado$Data <- seq.Date(from = min(dados$Data),by="7 days", 
                          length.out = nrow(esperado))
esperado$ISOweek <- ISOweek(esperado$Data)
tail(esperado)
```


#desenhar e estimativa


```{r}
plot(esperado$upr99~esperado$Data,col="red",type="l",ylim=c(0,2000),
     xlab="Semana",ylab="Obitos",las=1,axes=F,
     main="Mortalidade Semanal em Portugal \n CircResp")
axis.Date(1,at=seq(min(esperado$Data),max(esperado$Data)+1,by="6 months"),format="%V-%Y",cex.axis=0.6,las=2)
axis(2,cex.axis=0.6,las=1)
lines(esperado$upr95~esperado$Data,col="orange")
lines(esperado$fit~esperado$Data,col="blue")
lines(esperado$lwr95~esperado$Data,col="green")
lines(esperado$lwr99~esperado$Data,col="darkgreen")
lines(dados$CircResp~dados$Data)
```
